<a href="https://colab.research.google.com/github/jsansao/transformers_pt/blob/main/BERTimbau_Fine_tuning_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inferência usando transformers pré-treinados do HuggingFace 

In [ ]:
#@title Passo 1:Instalando Hugging Face Transformers
# We won't need TensorFlow here
!pip uninstall -y tensorflow
# Install `transformers` from master
!pip install transformers datasets
!pip list | grep -E 'transformers|tokenizers'
# transformers version at notebook update --- 2.9.1
# tokenizers version at notebook update --- 0.7.0

Found existing installation: tensorflow 2.8.2+zzzcolab20220929150707
Uninstalling tensorflow-2.8.2+zzzcolab20220929150707:
  Successfully uninstalled tensorflow-2.8.2+zzzcolab20220929150707
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 27.3 MB/s 
     |████████████████████████████████| 432 kB 56.9 MB/s 
     |████████████████████████████████| 163 kB 71.9 MB/s 
     |████████████████████████████████| 7.6 MB 63.2 MB/s 
     |████████████████████████████████| 115 kB 74.6 MB/s 
     |████████████████████████████████| 212 kB 75.3 MB/s 
     |████████████████████████████████| 127 kB 73.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the fol

In [ ]:
#@title Step 1: Loading the Dataset
#1.Load kant.txt using the Colab file manager
#2.Downloading the file from GitHub
!curl -L https://raw.githubusercontent.com/jsansao/corpus_ptbr/main/dump_Machado_Nobreak.txt --output "dump.txt"
#!curl -L https://raw.githubusercontent.com/jsansao/corpus_ptbr/main/Lyrics_ChicoBuarque.txt --output "kant.txt"
!awk NF < dump.txt > kant.txt


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1739k  100 1739k    0     0  1690k      0  0:00:01  0:00:01 --:--:-- 1691k


In [ ]:
#@title Passo 2:Baixando e salvando BR_BERTo
#https://huggingface.co/rdenadai/BR_BERTo

from transformers import AutoTokenizer, AutoModelForMaskedLM

#tokenizer = AutoTokenizer.from_pretrained("rdenadai/BR_BERTo")
#model = AutoModelForMaskedLM.from_pretrained("rdenadai/BR_BERTo")

tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')
model = AutoModelForMaskedLM.from_pretrained('neuralmind/bert-base-portuguese-cased')

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
!shuf -n1000 kant.txt > kant_teste.txt

In [ ]:
from datasets import load_dataset
datasets = load_dataset("text", data_files={"train": "kant.txt", "validation": "kant_teste.txt"})

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-8239196e210ed881/0.0.0/21a506d1b2b34316b1e82d0bd79066905d846e5d7e619823c0dd338d6f1fa6ad. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [ ]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

#0:   0%|          | 0/2 [00:00<?, ?ba/s]

#1:   0%|          | 0/2 [00:00<?, ?ba/s]

#2:   0%|          | 0/2 [00:00<?, ?ba/s]

#3:   0%|          | 0/2 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

exemplo de entrada

In [ ]:
tokenized_datasets["train"][4]

{'input_ids': [101, 1627, 391, 16581, 340, 125, 20229, 102],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
#block_size = tokenizer.model_max_length
block_size = 32


In [ ]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

#0:   0%|          | 0/2 [00:00<?, ?ba/s]

#1:   0%|          | 0/2 [00:00<?, ?ba/s]

#2:   0%|          | 0/2 [00:00<?, ?ba/s]

#3:   0%|          | 0/2 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
#@title Step 11: Defining a Data Collator
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [ ]:
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {metric.compute(predictions=predictions, references=labels)}

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
#@title Step 12: Initializing the Trainer
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./KantaiBERT",
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_device_train_batch_size=16,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],    
)

In [ ]:
#@title Step 13: Pre-training the Model
%%time
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 14821
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 9270


Step,Training Loss
500,2.651000
1000,2.470000
1500,2.347000
2000,2.321000
2500,2.194700
3000,2.145000
3500,2.130100
4000,2.082500
4500,2.021200
5000,1.973600




Training completed. Do not forget to share your model on huggingface.co/models =)




CPU times: user 20min 54s, sys: 1min 42s, total: 22min 36s
Wall time: 22min 35s


TrainOutput(global_step=9270, training_loss=2.0564137743771913, metrics={'train_runtime': 1355.7643, 'train_samples_per_second': 109.318, 'train_steps_per_second': 6.837, 'total_flos': 2438078365367040.0, 'train_loss': 2.0564137743771913, 'epoch': 10.0})

In [ ]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

***** Running Evaluation *****
  Num examples = 1949
  Batch size = 8


Perplexity: 6.23


In [ ]:
eval_results

{'eval_loss': 1.8296008110046387,
 'eval_runtime': 4.6816,
 'eval_samples_per_second': 416.31,
 'eval_steps_per_second': 52.119,
 'epoch': 10.0}

In [ ]:
#@title Step 14: Saving the Final Model(+tokenizer + config) to disk
trainer.save_model("./KantaiBERT")

Saving model checkpoint to ./KantaiBERT
Configuration saved in ./KantaiBERT/config.json
Model weights saved in ./KantaiBERT/pytorch_model.bin


In [ ]:
#@title Step 3: Configurando o pipeline fill-mask
#@title Step 15: Language Modeling with the FillMaskPipeline
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="./KantaiBERT",
    tokenizer=tokenizer
)

loading configuration file ./KantaiBERT/config.json
Model config BertConfig {
  "_name_or_path": "./KantaiBERT",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.23.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 29794
}

loading configuration file ./KantaiBERT/config.json
Model config BertConfig {
  "_nam

In [ ]:
fill_mask("O rapaz olhou para o [MASK] ")

[{'score': 0.2958708703517914,
  'token': 8242,
  'token_str': 'céu',
  'sequence': 'O rapaz olhou para o céu'},
 {'score': 0.1430724412202835,
  'token': 8105,
  'token_str': 'chão',
  'sequence': 'O rapaz olhou para o chão'},
 {'score': 0.09538926184177399,
  'token': 12639,
  'token_str': 'teto',
  'sequence': 'O rapaz olhou para o teto'},
 {'score': 0.040487658232450485,
  'token': 1341,
  'token_str': 'lado',
  'sequence': 'O rapaz olhou para o lado'},
 {'score': 0.03173784539103508,
  'token': 9186,
  'token_str': 'jardim',
  'sequence': 'O rapaz olhou para o jardim'}]

In [ ]:
fill_mask("A moça olhou para o [MASK] ")

[{'score': 0.3103584945201874,
  'token': 8242,
  'token_str': 'céu',
  'sequence': 'A moça olhou para o céu'},
 {'score': 0.1739509552717209,
  'token': 8105,
  'token_str': 'chão',
  'sequence': 'A moça olhou para o chão'},
 {'score': 0.06886520236730576,
  'token': 12639,
  'token_str': 'teto',
  'sequence': 'A moça olhou para o teto'},
 {'score': 0.038571085780858994,
  'token': 9186,
  'token_str': 'jardim',
  'sequence': 'A moça olhou para o jardim'},
 {'score': 0.03595158830285072,
  'token': 1341,
  'token_str': 'lado',
  'sequence': 'A moça olhou para o lado'}]

In [ ]:
fill_mask("Comprou uma [MASK] na loja. ")

[{'score': 0.2997158169746399,
  'token': 3743,
  'token_str': 'carta',
  'sequence': 'Comprou uma carta na loja.'},
 {'score': 0.12703746557235718,
  'token': 9104,
  'token_str': 'cadeira',
  'sequence': 'Comprou uma cadeira na loja.'},
 {'score': 0.11237221211194992,
  'token': 4428,
  'token_str': 'nota',
  'sequence': 'Comprou uma nota na loja.'},
 {'score': 0.04970420151948929,
  'token': 1105,
  'token_str': 'casa',
  'sequence': 'Comprou uma casa na loja.'},
 {'score': 0.0339292474091053,
  'token': 6890,
  'token_str': 'chave',
  'sequence': 'Comprou uma chave na loja.'}]

In [ ]:
fill_mask("A mulher não é [MASK]. ")

[{'score': 0.0728410854935646,
  'token': 1016,
  'token_str': 'assim',
  'sequence': 'A mulher não é assim.'},
 {'score': 0.07044544816017151,
  'token': 3874,
  'token_str': 'nada',
  'sequence': 'A mulher não é nada.'},
 {'score': 0.06973917037248611,
  'token': 19543,
  'token_str': 'bonita',
  'sequence': 'A mulher não é bonita.'},
 {'score': 0.06576164066791534,
  'token': 1766,
  'token_str': 'má',
  'sequence': 'A mulher não é má.'},
 {'score': 0.05279678851366043,
  'token': 2745,
  'token_str': 'tudo',
  'sequence': 'A mulher não é tudo.'}]

In [ ]:
fill_mask("O homem não é [MASK]. ")

[{'score': 0.0815165787935257,
  'token': 3874,
  'token_str': 'nada',
  'sequence': 'O homem não é nada.'},
 {'score': 0.08121395856142044,
  'token': 11775,
  'token_str': 'mau',
  'sequence': 'O homem não é mau.'},
 {'score': 0.07851838320493698,
  'token': 2397,
  'token_str': 'homem',
  'sequence': 'O homem não é homem.'},
 {'score': 0.03279002010822296,
  'token': 7255,
  'token_str': 'religioso',
  'sequence': 'O homem não é religioso.'},
 {'score': 0.03239091858267784,
  'token': 4062,
  'token_str': 'bom',
  'sequence': 'O homem não é bom.'}]

In [ ]:
fill_mask("Ele é um bom [MASK]. ")

[{'score': 0.2858264148235321,
  'token': 3695,
  'token_str': 'amigo',
  'sequence': 'Ele é um bom amigo.'},
 {'score': 0.20802903175354004,
  'token': 2397,
  'token_str': 'homem',
  'sequence': 'Ele é um bom homem.'},
 {'score': 0.09337740391492844,
  'token': 13254,
  'token_str': 'rapaz',
  'sequence': 'Ele é um bom rapaz.'},
 {'score': 0.07037695497274399,
  'token': 10110,
  'token_str': 'companheiro',
  'sequence': 'Ele é um bom companheiro.'},
 {'score': 0.04173014312982559,
  'token': 10738,
  'token_str': 'sujeito',
  'sequence': 'Ele é um bom sujeito.'}]

In [ ]:
fill_mask("Ela é uma boa [MASK]. ")

[{'score': 0.2570493221282959,
  'token': 8932,
  'token_str': 'amiga',
  'sequence': 'Ela é uma boa amiga.'},
 {'score': 0.11931397765874863,
  'token': 17704,
  'token_str': 'senhora',
  'sequence': 'Ela é uma boa senhora.'},
 {'score': 0.0853387638926506,
  'token': 2606,
  'token_str': 'mulher',
  'sequence': 'Ela é uma boa mulher.'},
 {'score': 0.08158737421035767,
  'token': 20216,
  'token_str': 'companheira',
  'sequence': 'Ela é uma boa companheira.'},
 {'score': 0.06872350722551346,
  'token': 2760,
  'token_str': 'pessoa',
  'sequence': 'Ela é uma boa pessoa.'}]

In [ ]:
fill_mask("Faz de conta que ainda é [MASK]. ")

[{'score': 0.26162201166152954,
  'token': 1373,
  'token_str': 'tarde',
  'sequence': 'Faz de conta que ainda é tarde.'},
 {'score': 0.09605362266302109,
  'token': 2397,
  'token_str': 'homem',
  'sequence': 'Faz de conta que ainda é homem.'},
 {'score': 0.0765121579170227,
  'token': 4964,
  'token_str': 'criança',
  'sequence': 'Faz de conta que ainda é criança.'},
 {'score': 0.05994915962219238,
  'token': 1695,
  'token_str': 'pouco',
  'sequence': 'Faz de conta que ainda é pouco.'},
 {'score': 0.041957639157772064,
  'token': 8545,
  'token_str': 'cedo',
  'sequence': 'Faz de conta que ainda é cedo.'}]

In [ ]:
fill_mask("Depois da tempestade vem a [MASK]. ")

[{'score': 0.22072544693946838,
  'token': 13943,
  'token_str': 'lua',
  'sequence': 'Depois da tempestade vem a lua.'},
 {'score': 0.18871186673641205,
  'token': 7658,
  'token_str': 'tempestade',
  'sequence': 'Depois da tempestade vem a tempestade.'},
 {'score': 0.08224523812532425,
  'token': 9856,
  'token_str': 'chuva',
  'sequence': 'Depois da tempestade vem a chuva.'},
 {'score': 0.04535399749875069,
  'token': 1386,
  'token_str': 'morte',
  'sequence': 'Depois da tempestade vem a morte.'},
 {'score': 0.04494915157556534,
  'token': 6293,
  'token_str': 'manhã',
  'sequence': 'Depois da tempestade vem a manhã.'}]

In [ ]:
fill_mask("Mais vale um pássaro na mão do que [MASK] voando. ")

[{'score': 0.2109585851430893,
  'token': 17804,
  'token_str': 'pássaros',
  'sequence': 'Mais vale um pássaro na mão do que pássaros voando.'},
 {'score': 0.19499671459197998,
  'token': 1342,
  'token_str': 'outro',
  'sequence': 'Mais vale um pássaro na mão do que outro voando.'},
 {'score': 0.15950024127960205,
  'token': 222,
  'token_str': 'um',
  'sequence': 'Mais vale um pássaro na mão do que um voando.'},
 {'score': 0.13960781693458557,
  'token': 14934,
  'token_str': 'voar',
  'sequence': 'Mais vale um pássaro na mão do que voar voando.'},
 {'score': 0.025587495416402817,
  'token': 925,
  'token_str': 'ir',
  'sequence': 'Mais vale um pássaro na mão do que ir voando.'}]

In [ ]:
fill_mask("Não tinha [MASK], mas almoçou mesmo assim. ")

[{'score': 0.07289928197860718,
  'token': 3874,
  'token_str': 'nada',
  'sequence': 'Não tinha nada, mas almoçou mesmo assim.'},
 {'score': 0.06720980256795883,
  'token': 11062,
  'token_str': 'fome',
  'sequence': 'Não tinha fome, mas almoçou mesmo assim.'},
 {'score': 0.06012691184878349,
  'token': 11334,
  'token_str': 'sono',
  'sequence': 'Não tinha sono, mas almoçou mesmo assim.'},
 {'score': 0.03574288636445999,
  'token': 2292,
  'token_str': 'filhos',
  'sequence': 'Não tinha filhos, mas almoçou mesmo assim.'},
 {'score': 0.03197363018989563,
  'token': 12215,
  'token_str': 'costume',
  'sequence': 'Não tinha costume, mas almoçou mesmo assim.'}]

In [ ]:
fill_mask("Bebeu um copo d'água, pois tinha [MASK]. ")

[{'score': 0.7914512753486633,
  'token': 14594,
  'token_str': 'febre',
  'sequence': "Bebeu um copo d'água, pois tinha febre."},
 {'score': 0.044778767973184586,
  'token': 11062,
  'token_str': 'fome',
  'sequence': "Bebeu um copo d'água, pois tinha fome."},
 {'score': 0.036608342081308365,
  'token': 2496,
  'token_str': 'sede',
  'sequence': "Bebeu um copo d'água, pois tinha sede."},
 {'score': 0.023047253489494324,
  'token': 11334,
  'token_str': 'sono',
  'sequence': "Bebeu um copo d'água, pois tinha sono."},
 {'score': 0.01965203694999218,
  'token': 7672,
  'token_str': 'medo',
  'sequence': "Bebeu um copo d'água, pois tinha medo."}]

In [ ]:
fill_mask("Sem saber, ele descobriu um [MASK]. ")

[{'score': 0.2890945374965668,
  'token': 11021,
  'token_str': 'segredo',
  'sequence': 'Sem saber, ele descobriu um segredo.'},
 {'score': 0.12827526032924652,
  'token': 16269,
  'token_str': 'mistério',
  'sequence': 'Sem saber, ele descobriu um mistério.'},
 {'score': 0.037165723741054535,
  'token': 3420,
  'token_str': 'caminho',
  'sequence': 'Sem saber, ele descobriu um caminho.'},
 {'score': 0.024419967085123062,
  'token': 14096,
  'token_str': 'desastre',
  'sequence': 'Sem saber, ele descobriu um desastre.'},
 {'score': 0.021394526585936546,
  'token': 3204,
  'token_str': 'plano',
  'sequence': 'Sem saber, ele descobriu um plano.'}]

In [ ]:
fill_mask("Pedro fez fortuna vendendo [MASK]. ")

[{'score': 0.1536049097776413,
  'token': 5976,
  'token_str': 'escravos',
  'sequence': 'Pedro fez fortuna vendendo escravos.'},
 {'score': 0.14707770943641663,
  'token': 2978,
  'token_str': 'livros',
  'sequence': 'Pedro fez fortuna vendendo livros.'},
 {'score': 0.07660498470067978,
  'token': 7568,
  'token_str': 'café',
  'sequence': 'Pedro fez fortuna vendendo café.'},
 {'score': 0.05529876798391342,
  'token': 20460,
  'token_str': 'doces',
  'sequence': 'Pedro fez fortuna vendendo doces.'},
 {'score': 0.049433063715696335,
  'token': 2987,
  'token_str': 'ouro',
  'sequence': 'Pedro fez fortuna vendendo ouro.'}]

In [ ]:
fill_mask("Sem medo de ser [MASK]. ")

[{'score': 0.9432085156440735,
  'token': 8540,
  'token_str': 'feliz',
  'sequence': 'Sem medo de ser feliz.'},
 {'score': 0.0030068824999034405,
  'token': 3382,
  'token_str': 'visto',
  'sequence': 'Sem medo de ser visto.'},
 {'score': 0.002647929824888706,
  'token': 7148,
  'token_str': 'padre',
  'sequence': 'Sem medo de ser padre.'},
 {'score': 0.0018947251373901963,
  'token': 2397,
  'token_str': 'homem',
  'sequence': 'Sem medo de ser homem.'},
 {'score': 0.0015609144465997815,
  'token': 4964,
  'token_str': 'criança',
  'sequence': 'Sem medo de ser criança.'}]